# Romanized Malayalam to Native Malayalam Reverse Tranliteration

In [99]:
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input, Embedding, GRU, Dense, RepeatVector, TimeDistributed, Concatenate, Dropout, LayerNormalization, Bidirectional, LSTM, GlobalAveragePooling1D, MultiHeadAttention
from keras.callbacks import EarlyStopping
from keras.layers import Embedding, Attention
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import nltk
from keras.regularizers import l2
import pandas as pd
from keras.optimizers import Adam
import tensorflow as tf
import keras.regularizers as regularizers

## 1. Defining tokens

### Defining English Characters

In [101]:
source_tokens = list('abcdefghijklmnopqrstuvwxyz ')
source_tokenizer = Tokenizer(char_level=True, filters='')
source_tokenizer.fit_on_texts(source_tokens)

### Defining Malayalam Characters

In [102]:
# Define Malayalam characters
malayalam_tokens = [
    # Independent vowels
    'അ', 'ആ', 'ഇ', 'ഈ', 'ഉ', 'ഊ', 'ഋ', 'ൠ', 'ഌ', 'ൡ', 'എ', 'ഏ', 'ഐ', 'ഒ', 'ഓ', 'ഔ',

    # Consonants
    'ക', 'ഖ', 'ഗ', 'ഘ', 'ങ', 'ച', 'ഛ', 'ജ', 'ഝ', 'ഞ',
    'ട', 'ഠ', 'ഡ', 'ഢ', 'ണ', 'ത', 'ഥ', 'ദ', 'ധ', 'ന',
    'പ', 'ഫ', 'ബ', 'ഭ', 'മ', 'യ', 'ര', 'ല', 'വ', 'ശ',
    'ഷ', 'സ', 'ഹ', 'ള', 'ഴ', 'റ',

    # Chillu letters
    'ൺ', 'ൻ', 'ർ', 'ൽ', 'ൾ',

    # Additional characters
    'ം', 'ഃ', '്',

    # Vowel modifiers / Signs
    'ാ', 'ി', 'ീ', 'ു', 'ൂ', 'ൃ', 'ൄ', 'െ', 'േ', 'ൈ', 'ൊ', 'ോ', 'ൌ', 'ൗ', ' '
]

# Create tokenizer for Malayalam tokens
target_tokenizer = Tokenizer(char_level=True, filters='')
target_tokenizer.fit_on_texts(malayalam_tokens)

## 2. Loading dataset from hugging face for training

### a) Dakshina romanized

In [120]:
from datasets import load_dataset, DatasetDict
dataset=DatasetDict()
#basilkr/Malasar_Dict_only
dataset = load_dataset("vrclc/Dakshina-romanized-ml",use_auth_token=True)

/jupyter/socse.user06/.local/lib/python3.9/site-packages/datasets/load.py:2479: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Generating train split:   0%|          | 0/185673 [00:00<?, ? examples/s]

In [6]:
dataset.shape

{'train': (185673, 2)}

In [10]:
data1=dataset['train']

In [11]:
data1.shape

(185673, 2)

In [14]:
data1['ml'][:10]

['ചാൾസ്',
 'തടാകം',
 '1742-ൽ',
 'എളയടത്തു',
 'സ്വരൂപത്തെ',
 'മാർത്താണ്ഡവർമ്മ',
 'തിരുവിതാംകൂറിൽ',
 'ലയിപ്പിച്ചു.',
 'ഡെന്മാർക്ക്',
 'ഗുരുവായൂർ']

## Remove empty spaces from dataset

#### To remove the identified empty rows in the dataset 

In [21]:
def filter_empty_rows(example):
    return (
        example['ml'] is not None and example['ml'].strip() != '' and
        example['en'] is not None and example['en'].strip() != ''
    )

# Apply the filter to the dataset
filtered_dataset = data1.filter(filter_empty_rows)

# Ensure 'ml' and 'en' are strings
def ensure_strings(example):
    example['ml'] = str(example['ml']) if example['ml'] is not None else ''
    example['en'] = str(example['en']) if example['en'] is not None else ''
    return example

# Map the transformation to the dataset
data1 = filtered_dataset.map(ensure_strings)

In [25]:
data1

Dataset({
    features: ['ml', 'en'],
    num_rows: 185608
})

## b) Aksharanthar dataset

In [26]:
# Load the dataset
dataset = load_dataset("vrclc/Aksharantar-ml",use_auth_token=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ml', 'en', 'source'],
        num_rows: 4100621
    })
    validation: Dataset({
        features: ['ml', 'en', 'source'],
        num_rows: 7613
    })
    test: Dataset({
        features: ['ml', 'en', 'source'],
        num_rows: 12451
    })
})

In [28]:
data2=dataset['train']

In [29]:
data2.shape

(4100621, 3)

## Remove empty spaces from dataset

In [30]:
filtered_dataset = data2.filter(filter_empty_rows)
data2 = filtered_dataset.map(ensure_strings)

Filter:   0%|          | 0/4100621 [00:00<?, ? examples/s]

Map:   0%|          | 0/4100621 [00:00<?, ? examples/s]

In [31]:
data2

Dataset({
    features: ['ml', 'en', 'source'],
    num_rows: 4100621
})

## Remove irrelevant columns

In [42]:
data2=data2.remove_columns(["source"])

In [43]:
data2

Dataset({
    features: ['ml', 'en'],
    num_rows: 4100621
})

## c) Dekshina Lexicon

In [ ]:
# Load the dataset
dataset = load_dataset("vrclc/dakshina-lexicons-ml",use_auth_token=True)

/jupyter/socse.user06/.local/lib/python3.9/site-packages/datasets/load.py:2479: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [34]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ml', 'en', 'count'],
        num_rows: 58382
    })
    validation: Dataset({
        features: ['ml', 'en', 'count'],
        num_rows: 5641
    })
    test: Dataset({
        features: ['ml', 'en', 'count'],
        num_rows: 5610
    })
})

In [35]:
data3=dataset['train']

In [36]:
data3

Dataset({
    features: ['ml', 'en', 'count'],
    num_rows: 58382
})

## Remove empty spaces from dataset

In [38]:
filtered_dataset = data3.filter(filter_empty_rows)
data3 = filtered_dataset.map(ensure_strings)

Filter:   0%|          | 0/58382 [00:00<?, ? examples/s]

Map:   0%|          | 0/58381 [00:00<?, ? examples/s]

In [41]:
data3=data3.remove_columns(["count"])

In [44]:
data3

Dataset({
    features: ['ml', 'en'],
    num_rows: 58381
})

## Three dataets

In [45]:
data1[0:10]

{'ml': ['ചാൾസ്',
  'തടാകം',
  '1742-ൽ',
  'എളയടത്തു',
  'സ്വരൂപത്തെ',
  'മാർത്താണ്ഡവർമ്മ',
  'തിരുവിതാംകൂറിൽ',
  'ലയിപ്പിച്ചു.',
  'ഡെന്മാർക്ക്',
  'ഗുരുവായൂർ'],
 'en': ['Charles',
  'thadakom',
  '1742-il',
  'Ilayidathu',
  'Swarupathe',
  'Marthandavarma',
  'Thiruvithamkooril',
  'layipichu.',
  'Denmark',
  'Guruvayur']}

In [46]:
data2[0:10]

{'ml': ['തൈകളുടെ',
  'ആഘാതം',
  'തമ്മിലുണ്ടായ',
  'മാറ്റിയെടുത്തു',
  'സ്വന്തമായി',
  'അപകടകാരികളായ',
  'ശരിയാണെന്ന്',
  'ചാന്ദ്',
  'ദേവിയാണ്',
  'നാമമാണ്'],
 'en': ['thykalude',
  'aakhaatham',
  'thammillundaaya',
  'mattiyeduthu',
  'svanthamaayi',
  'apakadakarikalaaya',
  'shariyaanennu',
  'chaandu',
  'dheviyaan',
  'naamamaanu']}

In [47]:
data3[0:10]

{'ml': ['അം',
  'അംഗ',
  'അംഗ',
  'അംഗം',
  'അംഗം',
  'അംഗങ്ങളാണ്',
  'അംഗങ്ങളാണ്',
  'അംഗങ്ങളായ',
  'അംഗങ്ങളായ',
  'അംഗങ്ങളായി'],
 'en': ['am',
  'amga',
  'anga',
  'amgam',
  'angam',
  'amgangalaanu',
  'angangalaanu',
  'amgangalaaya',
  'angangalaaya',
  'amgangaayi']}

## Combine dataset

In [48]:
from datasets import concatenate_datasets

In [49]:
# Combine the datasets
data = concatenate_datasets([data1, data2, data3])

In [50]:
data

Dataset({
    features: ['ml', 'en'],
    num_rows: 4344610
})

## 3. Data cleaning - case change, remove all undefined tokens (numbers, puntuations etc..), 

In [51]:
def clean_text(example):
    example['en'] = example['en'].lower()  # Convert to lowercase
    cleaned_en = ''.join([' ' if char not in source_tokens else char for char in example['en']]).strip()
    cleaned_en = ' '.join(cleaned_en.split())  # Replace multiple spaces with single
    example['en'] = cleaned_en
    cleaned_ml = ''.join([' ' if char not in malayalam_tokens else char for char in example['ml']]).strip()
    cleaned_ml = ' '.join(cleaned_ml.split())  # Replace multiple spaces with single
    example['ml'] = cleaned_ml
    return example

# Apply the cleaning functions to the dataset
cleaned_dataset = data.map(clean_text)

Map:   0%|          | 0/4344610 [00:00<?, ? examples/s]

In [53]:
cleaned_dataset[0:20]

{'ml': ['ചാൾസ്',
  'തടാകം',
  'ൽ',
  'എളയടത്തു',
  'സ്വരൂപത്തെ',
  'മാർത്താണ്ഡവർമ്മ',
  'തിരുവിതാംകൂറിൽ',
  'ലയിപ്പിച്ചു',
  'ഡെന്മാർക്ക്',
  'ഗുരുവായൂർ',
  '',
  'പട്ടാമ്പി',
  'പൊന്നാനി',
  '',
  'പാലക്കാട്',
  'പാതയിലെ',
  'ഒരു',
  'സുപ്രധാന',
  'പട്ടണവും',
  'തൃത്താല'],
 'en': ['charles',
  'thadakom',
  'il',
  'ilayidathu',
  'swarupathe',
  'marthandavarma',
  'thiruvithamkooril',
  'layipichu',
  'denmark',
  'guruvayur',
  '',
  'pattambi',
  'ponnani',
  '',
  'palakad',
  'paathayile',
  'oru',
  'supradhana',
  'pattanavum',
  'thrithala']}

In [56]:
cleaned_dataset.shape

(4344610, 2)

## Remove empty rows

In [54]:
filtered_dataset = cleaned_dataset.filter(filter_empty_rows)

Filter:   0%|          | 0/4344610 [00:00<?, ? examples/s]

In [58]:
filtered_dataset[100000:100010]

{'ml': ['ഗിന്നസ്',
  'റെക്കോഡും',
  'ഈ',
  'സെന്റിമീറ്ററുകാരനെ',
  'തേടിയെത്തിയിരുന്നു',
  'നവഭവദശോകദളശയനസാരേ',
  'സൂപ്പർടക്സ്കാർട്ടിന്റെ',
  'ഗെയിംപ്ലേ',
  'മരിയോ',
  'കാർട്ട്'],
 'en': ['guiness',
  'recordum',
  'ee',
  'centimeterukarane',
  'thediyethiyirunnu',
  'navabhavadhashokadhalashayanasare',
  'sooppartakskaarttinte',
  'gaiyimple',
  'mariyo',
  'kaart']}

In [57]:
filtered_dataset.shape

(4342136, 2)

In [59]:
data=filtered_dataset

## 4. To fix the maximum input sequence length

In [61]:
# Calculate the length of 'ml' and 'en' fields
def calculate_lengths(example):
    return {
        'ml_length': len(example['ml']),
        'en_length': len(example['en'])
    }

# Apply the function to calculate lengths
lengths_dataset = data.map(calculate_lengths)

# Extract the maximum length for both fields
max_ml_length = max(lengths_dataset['ml_length'])
max_en_length = max(lengths_dataset['en_length'])

# Calculate the maximum sequence length overall
max_seq_length = max(max_ml_length, max_en_length)

print(max_seq_length)

Map:   0%|          | 0/4342136 [00:00<?, ? examples/s]

57


In [62]:
print(max_ml_length)
print(max_en_length)

50
57


## set input lenth to max_seq_length by padding

In [63]:
source_sequences = source_tokenizer.texts_to_sequences(data['en'])
X = pad_sequences(source_sequences, maxlen=max_seq_length, padding='post')

target_sequences = target_tokenizer.texts_to_sequences(data['ml'])
y = pad_sequences(target_sequences, maxlen=max_seq_length, padding='post')

In [66]:
y.shape

(4342136, 57)

## 5. Train - Validation data split

In [71]:
# Train-Validation-Test Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [72]:
X_train.shape

(3907922, 57)

In [73]:
X_val.shape

(434214, 57)

## 6. Model Defenition 

In [75]:
# Define the model parameters
embedding_dim = 64
hidden_units = 128  # Adjusted to match both encoder and decoder
#max_seq_length = max(data['ml'].apply(len).max(), data['en'].apply(len).max())

# Encoder
encoder_input = Input(shape=(max_seq_length,), name='encoder_input')
encoder_embedding = Embedding(input_dim=len(source_tokenizer.word_index) + 1, output_dim=embedding_dim, name='encoder_embedding')(encoder_input)
encoder_lstm = Bidirectional(LSTM(hidden_units, return_sequences=True, name='encoder_lstm'))(encoder_embedding)

# Adjust encoder output for attention compatibility
encoder_lstm_adjusted = Dense(hidden_units, activation='relu', name='encoder_dense_adjust')(encoder_lstm)

# Decoder
decoder_input = RepeatVector(max_seq_length, name='decoder_repeat_vector')(encoder_lstm_adjusted[:, -1, :])  # Use last state for RepeatVector
decoder_lstm = LSTM(hidden_units, return_sequences=True, name='decoder_lstm')(decoder_input)

# Apply attention between the encoder outputs and decoder outputs
attention_output = Attention(name='attention_layer')([decoder_lstm, encoder_lstm_adjusted])
decoder_concat = Concatenate(axis=-1, name='decoder_concat')([decoder_lstm, attention_output])
decoder_dense = TimeDistributed(Dense(len(target_tokenizer.word_index) + 1, activation='softmax', name='decoder_output_dense'))(decoder_concat)

# Define the model
model = Model(inputs=encoder_input, outputs=decoder_dense)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


2024-12-05 14:08:26.271098: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [77]:
# Set up callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('leenag/transliteration', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

### 7. Training 

In [ ]:
history = model.fit(
    X_train, np.expand_dims(y_train, -1),
    validation_data=(X_val, np.expand_dims(y_val, -1)),
    epochs=10,
    batch_size=64,
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)


Epoch 1/10
39969/61062 [==================>...........] - ETA: 39:09 - loss: 0.1248 - accuracy: 0.9610

In [ ]:
# '''  ''' Plot training history
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

### 8. Save model

In [80]:
# Assuming you are using a Keras model
model.save('leenag/transliteration')


INFO:tensorflow:Assets written to: leenag/transliteration/assets


INFO:tensorflow:Assets written to: leenag/transliteration/assets


In [81]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_input (InputLayer)  [(None, 57)]                 0         []                            
                                                                                                  
 encoder_embedding (Embeddi  (None, 57, 64)               1792      ['encoder_input[0][0]']       
 ng)                                                                                              
                                                                                                  
 bidirectional (Bidirection  (None, 57, 256)              197632    ['encoder_embedding[0][0]']   
 al)                                                                                              
                                                                                              

## 9. Load Model for Testing

In [82]:
from keras.models import load_model
# Load the saved model
model = load_model('leenag/transliteration')

### Testing with a romanized input

In [95]:
# prompt: Rewrite the above code to split the input_text by tokens and non-tokens. Pass the tokens to the model and get the output, retain the non-tokens as such in the output

import re

def transliterate_with_non_tokens(input_text, model, source_tokenizer, target_tokenizer, max_seq_length):
    """
    Transliterates input text, preserving non-token characters.
    """
    # Regular expression to split the text into tokens and non-tokens
    tokens_and_non_tokens = re.findall(r"([a-zA-Z]+)|([^a-zA-Z]+)", input_text)

    transliterated_text = ""
    for token_or_non_token in tokens_and_non_tokens:
        token = token_or_non_token[0]
        non_token = token_or_non_token[1]

        if token:
            input_sequence = source_tokenizer.texts_to_sequences([token])[0]
            input_sequence_padded = pad_sequences([input_sequence], maxlen=max_seq_length, padding='post')
            predicted_sequence = model.predict(input_sequence_padded)
            predicted_indices = np.argmax(predicted_sequence, axis=-1)[0]
            transliterated_word = ''.join([target_tokenizer.index_word[idx] for idx in predicted_indices if idx != 0])
            transliterated_text += transliterated_word
        elif non_token:
            transliterated_text += non_token

    return transliterated_text

In [122]:
input_text = "prabhathathile urakkathinu oru vallaatha sukhamanu"
transliterated_text = transliterate_with_non_tokens(input_text, model, source_tokenizer, target_tokenizer, max_seq_length)
print("Predicted Transliteration:", transliterated_text)

1/1 [==============================] - 0s 38ms/step
Predicted Transliteration: പ്രഭാതത്തിലെ ഉറക്കത്തിന് ഒരു വല്ലാത്ത സുഖമാണ്
